In [1]:
from SpinTrain import *

Nz = Operator('N','S^z_{N}')
az = [Operator('a%d'%i, 'S^z_{a%d}'%i) for i in range(3)]
bz = [Operator('b%d'%i, 'S^z_{b%d}'%i) for i in range(3)]
cz = [Operator('c%d'%i, 'S^z_{c%d}'%i) for i in range(3)]
nz = [Operator('n%d'%i, 'S^z_{n%d}'%i) for i in range(3)]

a_up = [Operator('a⁺%d'%i, 'S^+_{a%d}'%i) for i in range(3)]
a_dn = [Operator('a⁻%d'%i, 'S^-_{a%d}'%i) for i in range(3)]

ca = CommutatorAlgebra()

def define_spinalg(plus, minus, z):
    ca.set_commutator(plus, minus)(2*z)
    ca.set_commutator(z, plus)( plus )
    ca.set_commutator(z, minus)( -1*minus )

for i in range(3):
    define_spinalg(a_up[i],a_dn[i], az[i])
    ca.add_operator(bz[i])
    ca.add_operator(nz[i])
    ca.add_operator(cz[i])
    
    
ca.add_operator(Nz)

In [2]:
# define the Z objects

bond_zetas = [Operator('ζ%d'%i,r'\zeta_%d'%i,scalar=True) for i in range(3)]
bond_zetas_bar = [Operator('ζb%d'%i,r'\zeta_%d^*'%i,scalar=True) for i in range(3)]

KA = Operator('KA','K_A',scalar=True)
KB = Operator('KB','K_B',scalar=True)

def conjugate_zetas(x):
    repl = []
    for i in range(3):
        repl.append((bond_zetas[i], bond_zetas_bar[i]))
        repl.append((bond_zetas_bar[i], bond_zetas[i]))
    return x.replaceall(*repl)


Z = [0,0,0]

for i in range(3):
    ip = (i+1)%3
    ipp = (i+2)%3
    Z[i] = KA*(bond_zetas[i]*Nz + bond_zetas[ip]* az[ipp] + bond_zetas[ipp]*az[ip]) + KB*(bond_zetas[i]*nz[i] + bond_zetas[ip]*cz[i] + bond_zetas[ipp]*bz[i])

In [3]:


def shift_ops_right(x):
    x.move_scalars()
    for i in range(3):
        ca.move_right(x,az[i])
        ca.move_right(x,bz[i])
        ca.move_right(x,cz[i])
        ca.move_right(x,nz[i])
    
    ca.move_right(x,Nz)
    
    for i in range(3):
        ca.move_right(x,a_up[i])
        ca.move_right(x,a_dn[i])
    x.collect()
    
    
def shift_zs_left(x):
    for i in range(3):
        ca.move_left(x,az[i])
        ca.move_left(x,bz[i])
        ca.move_left(x,cz[i])
        ca.move_left(x,nz[i])
        
    for i in range(3):
        ca.move_left(x,Zs[i])
        ca.move_left(x,Zs_bar[i])

    
    ca.move_left(x,Nz)
    order_scalars(x)
    x.collect()
        
def order_scalars(x):
    canon_order = [KA,KB] + bond_zetas + bond_zetas_bar
    for s in reversed(canon_order):
        ca.move_left(x, s)

        
        
        
# build a new commutator database for only the ladders and the Z's

def calc_commutator(x1, x2):
    x = x1*x2 + x2 * x1 * -1
    order_scalars(x)
    shift_ops_right(x)
    x.collect()
    return x

    
    
# define the **symbolic** z's and zbars
Zs = [Operator('Z%d'%i,'Z_{a%d}'%i) for i in range(3)]
Zs_bar = [Operator('Z_bar%d'%i,'Z^*_{a%d}'%i) for i in range(3)]

for i in range(3):
    for j in range(3):
        # re-express the knowledge in terms of symbol Zs
        ca.set_commutator(Zs[j], a_up[i])(calc_commutator(Z[j],a_up[i]))
        ca.set_commutator(Zs[j], a_dn[i])(calc_commutator(Z[j],a_dn[i]))
        
        ca.set_commutator(Zs_bar[j], a_up[i])(calc_commutator(conjugate_zetas(Z[j]),a_up[i]))
        ca.set_commutator(Zs_bar[j], a_dn[i])(calc_commutator(conjugate_zetas(Z[j]),a_dn[i]))
        
        
        assert calc_commutator(Z[i],az[i]) == 0
        assert calc_commutator(conjugate_zetas(Z[i]),az[i]) == 0

In [4]:
sides = []

for i in range(3):
    ip = (i+1)%3
    ipp = (i+2)%3
    
    h12 = a_up[ip]*a_dn[i]
    kp = Zs[i]*a_up[i]
    km = Zs_bar[ip]*a_dn[ip]
    part = h12*kp*km + km*h12*kp + kp*km*h12 + h12*km*kp + km*kp*h12 + kp*h12*km
    
    h12 = a_dn[ip]*a_up[i]
    kp = Zs_bar[i]*a_dn[i]
    km = Zs[ip]*a_up[ip]
    part += h12*kp*km + km*h12*kp + kp*km*h12 + h12*km*kp + km*kp*h12 + kp*h12*km
    
    sides.append(part)
#     order_scalars(sides[i])
    shift_zs_left(sides[i])

    

In [5]:
expr = sides[0] + sides[1] + sides[2]

expr.collect()
expr.show()



<IPython.core.display.Latex object>

In [60]:
# simplify using the anticommutators
ca.move_right(expr,a_up[0])
ca.move_right(expr,a_dn[0])
ca.move_right(expr,a_up[1])
ca.move_right(expr,a_dn[1])
ca.move_right(expr,a_up[2])
ca.move_right(expr,a_dn[2])
expr.collect()



s = expr.replaceall(*[(a_up[i]*a_dn[i], az[i]+Fraction(1,2)) for i in range(3)])
shift_zs_left(s)
s2 = s.replaceall(*[(Zs[i], Z[i]) for i in range(3)]).replaceall(*[(Zs_bar[i], conjugate_zetas(Z[i])) for i in range(3)])
shift_ops_right(s2)
s2.collect()

In [61]:
w = Operator('w','w',scalar=True)
w_bar = Operator('wb',r'w^*',scalar=True)

def simplify_zetas(expr):
    # only works if scalars are in canonical order
    simpl = []
    for i, z in enumerate(bond_zetas):
        for j, zb in enumerate(bond_zetas_bar):
            if i == j:
                simpl.append((z*zb, 1))
            elif i == (j+1) %3:
                simpl.append((z*zb, w))
            else:
                simpl.append((z*zb, w_bar))
    return expr.replaceall(*simpl).replaceall((w*w_bar, 1), (w_bar*w, 1))

for i in range(3):
    ca.move_left(s2, bond_zetas_bar[i])
for i in range(3):
    ca.move_left(s2, bond_zetas[i])
    
s3 = simplify_zetas(s2)
ca.move_right(s3,w)
ca.move_right(s3,w_bar)

shift_ops_right(s3)

s4 = s3.replaceall( *[(az[i]*az[i], Fraction(1,4)) for i in range(3)]
    ).replaceall(   *[(bz[i]*bz[i], Fraction(1,4)) for i in range(3)]
    ).replaceall(   *[(cz[i]*cz[i], Fraction(1,4)) for i in range(3)]
    ).replaceall(   *[(nz[i]*nz[i], Fraction(1,4)) for i in range(3)]
    ).replaceall((Nz*Nz, Fraction(1,4)))

order_scalars(s4)

s4.collect()

In [62]:
ca.move_right(s4,Nz)
coeff = s4.coefficient(Nz)

In [63]:
root3 = Operator('√3',r'\sqrt{3}',scalar=True)

def convert_to_rectangular(expr):
    ca.move_right(expr, w)
    ca.move_right(expr, w_bar)
    coeff_w=expr.coefficient(w)
    coeff_wb=expr.coefficient(w_bar)
    
    remainder = expr + -1*(coeff_w*w +coeff_wb*w_bar)
    remainder.collect()
    real = remainder + Fraction(-1,2)*(coeff_w + coeff_wb)
    imag = Fraction(1,2)*(coeff_w + -1*coeff_wb)*root3
    return real, imag


re, im = convert_to_rectangular(coeff)

im.collect()
im.show()
re.collect()
re.show()

# Even with anisotropies, no complex components!

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [64]:
front = re.coefficient(KA*KA,'right')
front.show()

# this is a constant on the ice manifold!

<IPython.core.display.Latex object>

In [65]:
back = re.coefficient(KB*KB,'right')
back.show()
# sweet bro

<IPython.core.display.Latex object>

In [57]:
middle = re.coefficient(KA*KB,'right')
middle.sort('multiplier')
middle.show()

<IPython.core.display.Latex object>

In [ ]:
coeff_N_simpl = Fraction(-3,2)*(nz[0]+nz[1]+nz[2])
for i in range(3):
    ip = (i+1)%3
    ipp = (i+2)%3
    coeff_N_simpl += 6*az[i]*az[ip]*(n[i]+n[ip])
    
    
